In [3]:
import pandas as pd
import missingno as msno
import scipy.stats as stats
import numpy as np
from random import sample
import os
import matplotlib
#matplotlib.use('Agg')
import matplotlib.pyplot as plt

In [4]:
PREFIX_DATA_DIR = '/home/gernot/Nextcloud/Research/Papers/20_Political_Prediction/Data_Analysis/data/' 

In [5]:
CHES2019 = pd.read_stata(PREFIX_DATA_DIR + 'CHES2019_experts.dta')
CHES2019_means = pd.read_stata(PREFIX_DATA_DIR + 'CHES2019_means.dta')
EVS = pd.read_csv(PREFIX_DATA_DIR + 'voter_table.csv')
#EES = pd.read_stata('/home/gernot/pCloud/University_Graz/Research/Papers/20_Political_Prediction/Data_Analysis/data/EES_Voter_Survey.dta')

In [14]:
EVS.describe()

,respid,hCountry,countrycode,Q9_ees_numeric,lrgen_selfdescription,econ_interven,redistribution,sociallifestyle,civlib_laworder,immigrate_policy_reversed,...,prediction_lm_lrecon1,prediction_polr_lrecon1,prediction_rf_lrecon1,prediction_rf_lrecon2,prediction_rf_lrecon2_unrounded,prediction_lm_galtan1,prediction_polr_galtan1,prediction_rf_galtan1,prediction_rf_galtan2,prediction_rf_galtan2_unrounded
count,26538.000000,26538.000000,26538.000000,1.857600e+04,22826.000000,23954.000000,24907.000000,25543.000000,25204.000000,25124.000000,...,23232.000000,23232.000000,23232.000000,23232.000000,23232.000000,24078.000000,24465.000000,24078.000000,24078.000000,24078.000000
mean,28373.554865,14.518502,1405.441405,1.409567e+06,5.216376,5.014361,4.360903,4.563090,4.910887,4.398583,...,4.964575,5.006930,4.699337,4.886751,4.872369,5.025500,4.846025,4.877897,4.931971,4.925010
std,38690.502155,8.164329,225.340215,2.294313e+05,2.593870,2.596487,2.977558,4.000323,3.071499,3.274311,...,1.893327,2.040504,2.308023,1.844582,1.819117,2.412228,2.687408,3.678852,2.433879,2.409955
min,1.000000,1.000000,1040.000000,1.040110e+06,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,1.000000,1.000000,0.000000,1.000000,0.997203,0.000000,0.000000,0.000000,1.000000,0.652800
25%,7525.250000,8.000000,1233.000000,1.233411e+06,4.000000,3.000000,2.000000,0.000000,3.000000,1.000000,...,4.000000,4.000000,3.000000,4.000000,3.546396,3.000000,3.000000,1.000000,3.000000,2.801393
50%,24381.500000,14.000000,1372.000000,1.372951e+06,5.000000,5.000000,5.000000,5.000000,5.000000,5.000000,...,5.000000,5.000000,5.000000,5.000000,4.876749,5.000000,5.000000,5.000000,5.000000,4.947881
75%,37695.500000,22.000000,1620.000000,1.620311e+06,7.000000,7.000000,6.000000,9.000000,7.000000,7.000000,...,6.000000,7.000000,6.000000,6.000000,6.127804,7.000000,7.000000,9.000000,7.000000,7.020766
max,284138.000000,28.000000,1826.000000,1.826951e+06,10.000000,10.000000,10.000000,10.000000,10.000000,10.000000,...,10.000000,10.000000,10.000000,9.000000,9.294161,10.000000,10.000000,10.000000,10.000000,9.538374


In [9]:
relevant_cols = ['econ_interven', 
               'environment', 
               'redistribution', 
               'civlib_laworder', 
               'immigrate_policy', 
               'sociallifestyle',
               'lrgen',
               'lrecon',
               'galtan']

relevant_cols_ext = relevant_cols
relevant_cols_ext.append('party_id')

relevant_cols_independent = ['econ_interven', 
               'environment', 
               'redistribution', 
               'civlib_laworder', 
               'immigrate_policy', 
               'sociallifestyle']

relevant_cols_dependent = ['lrgen',
               'lrecon',
               'galtan']

relevant_cols_factor = ['econ_interven', 
               'environment', 
               'redistribution', 
               'civlib_laworder', 
               'immigrate_policy', 
               'sociallifestyle',
               'lrgen_factor',
               'lrecon_factor',
               'galtan_factor']

relevant_cols_factor_independent = ['econ_interven', 
               'environment', 
               'redistribution', 
               'civlib_laworder', 
               'immigrate_policy', 
               'sociallifestyle']

relevant_cols_factor_dependent = [
               'lrgen_factor',
               'lrecon_factor',
               'galtan_factor']

relevant_cols_EVS = ['econ_interven', 
               'environment', 
               'redistribution', 
               'civlib_laworder', 
               'immigrate_policy', 
               'sociallifestyle',
               'lrgen_selfdescription',
               'respid',
               'hCountry',
               'countrycode']

relevant_cols_EVS_independent = ['econ_interven', 
               'environment', 
               'redistribution', 
               'civlib_laworder', 
               'immigrate_policy', 
               'sociallifestyle']

# CHES2019

In [5]:
CHES2019.head()

,id,party,party_name,party_id,position,eu_salience,eu_dissent,eu_blur,lrecon,lrecon_blur,...,dob,lrecon_self,galtan_self,cname,eu_position_sd,lrecon_sd,galtan_sd,eu_econ_require,eu_political_require,eu_googov_require
0,1.0,1,GERB,2010.0,7.0,9.0,1.0,NaN,5.0,NaN,...,1948.0,9.0,3.0,,NaN,NaN,NaN,NaN,NaN,NaN
1,1.0,2,BSP,2003.0,5.0,6.0,4.0,NaN,3.0,NaN,...,1948.0,9.0,3.0,,NaN,NaN,NaN,NaN,NaN,NaN
2,1.0,3,DPS,2004.0,7.0,9.0,2.0,NaN,5.0,NaN,...,1948.0,9.0,3.0,,NaN,NaN,NaN,NaN,NaN,NaN
3,1.0,4,Volya,2017.0,3.0,6.0,4.0,NaN,7.0,NaN,...,1948.0,9.0,3.0,,NaN,NaN,NaN,NaN,NaN,NaN
4,1.0,5,NFSB,2014.0,3.0,6.0,4.0,NaN,7.0,NaN,...,1948.0,9.0,3.0,,NaN,NaN,NaN,NaN,NaN,NaN


In [15]:
CHES2019[relevant_cols_ext].info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3823 entries, 0 to 3822
Data columns (total 10 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   econ_interven     3432 non-null   float64
 1   environment       3262 non-null   float64
 2   redistribution    3396 non-null   float64
 3   civlib_laworder   3405 non-null   float64
 4   immigrate_policy  3438 non-null   float64
 5   sociallifestyle   3461 non-null   float64
 6   lrgen             3610 non-null   float64
 7   lrecon            3554 non-null   float64
 8   galtan            3596 non-null   float64
 9   party_id          3823 non-null   float32
dtypes: float32(1), float64(9)
memory usage: 283.9 KB


In [8]:
CHES2019[relevant_cols_ext].isnull().any()

econ_interven       True
environment         True
redistribution      True
civlib_laworder     True
immigrate_policy    True
sociallifestyle     True
lrgen               True
lrecon              True
galtan              True
dtype: bool

In [9]:
CHES2019[relevant_cols_ext].isnull().sum()

econ_interven       391
environment         561
redistribution      427
civlib_laworder     418
immigrate_policy    385
sociallifestyle     362
lrgen               213
lrecon              269
galtan              227
dtype: int64

In [16]:
# Obtain an overview of missing values and respective removal
data_base = CHES2019[relevant_cols_ext]
for i in range(0, 7):
    print(str(i) + ': ' + str(len(data_base.dropna(subset = data_base.iloc[:,:-4].columns.values,thresh = i))))

0: 3823
1: 3649
2: 3599
3: 3535
4: 3425
5: 3274
6: 2912


In [18]:
data_base.dropna(subset = data_base.iloc[:,:-4].columns.values,thresh = 6).to_csv(PREFIX_DATA_DIR + 'CHES2019_incl_party_id_without_imputation_all_items_filled.csv')

# EVS

In [12]:
data_base_EVS = EVS[relevant_cols_EVS_independent]

In [15]:
data_base_EVS.dropna(subset = data_base_EVS.columns.values,thresh = 6).to_csv(PREFIX_DATA_DIR + 'EVS_without_imputation_all_items_filled.csv',index=False)